# VacationPy
----

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
weather_data = pd.read_csv("./output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Saint Paul Harbor,90,US,1619913820,61,57.7900,-152.4072,46.40,12.66
1,Mahébourg,40,MU,1619913820,94,-20.4081,57.7000,73.40,3.44
2,Harper,100,LR,1619913821,81,4.3750,-7.7169,80.46,7.45
3,Mossamedes,19,AO,1619913821,78,-15.1961,12.1522,76.53,5.30
4,Kaeo,0,NZ,1619913821,63,-35.1000,173.7833,66.99,4.00
...,...,...,...,...,...,...,...,...,...
569,Sarti,95,GR,1619913993,76,40.1000,23.9833,63.72,5.28
570,Itaituba,100,BR,1619913993,97,-4.2761,-55.9836,74.50,1.79
571,Wawa,96,NG,1619913994,54,9.9022,4.4192,84.94,11.16
572,Ouésso,94,CG,1619913994,98,1.6136,16.0517,70.57,2.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)

In [27]:
# Plot Heatmap
fig = gmaps.figure(center=(15.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
max_intensity

100.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Narrow the cities with wind speed less yhaan 15 mph, cloudiness less than/equal to 10 and temperature range of 70-85

narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 7) & (weather_data["Cloudiness"] <= 5) & 
                                    (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 85)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
31,Cabo San Lucas,1,MX,1619913713,47,22.8909,-109.9124,78.80,6.91
56,Milledgeville,1,US,1619913837,53,33.0801,-83.2321,72.00,4.03
64,Saint-Philippe,0,RE,1619913839,73,-21.3585,55.7679,73.40,5.75
75,Atuona,2,PF,1619913816,68,-9.8000,-139.0333,79.39,6.82
120,Bogalusa,1,US,1619913858,44,30.7910,-89.8487,77.00,6.91
134,Lumberton,1,US,1619913862,30,34.6182,-79.0086,71.60,3.44
192,Prome,1,MM,1619913878,68,18.8167,95.2167,81.37,4.27
196,Shangrao,0,CN,1619913880,72,28.4532,117.9686,74.77,5.39
258,Statesboro,1,US,1619913900,49,32.4488,-81.7832,73.40,5.75
299,Pozo Colorado,0,PY,1619913912,48,-23.4923,-58.7972,73.26,5.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
# Create a hotel DataFrame
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
31,Cabo San Lucas,MX,22.8909,-109.9124,
56,Milledgeville,US,33.0801,-83.2321,
64,Saint-Philippe,RE,-21.3585,55.7679,
75,Atuona,PF,-9.8000,-139.0333,
120,Bogalusa,US,30.7910,-89.8487,
134,Lumberton,US,34.6182,-79.0086,
192,Prome,MM,18.8167,95.2167,
196,Shangrao,CN,28.4532,117.9686,
258,Statesboro,US,32.4488,-81.7832,
299,Pozo Colorado,PY,-23.4923,-58.7972,


In [33]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
             "keyword": "hotel",
             "radius": 5000,
             "key": g_key}

In [34]:
for index, row in hotel_df.iterrows():
    # Get city name, lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # Add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    print(f"Retrieving results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response["results"]
    
    # Save the hotel name to DataFrame
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    # If no hotel is available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("--------------------------")
    
    # Wait 1 second to make another API request to avoid SSL Error
    time.sleep(1)
    
# Print end of search after complete
print("-------End of Search------")

Retrieving results for Index 31: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Bahia Hotel & Beach House.
--------------------------
Retrieving results for Index 56: Milledgeville.
Closest hotel in Milledgeville is Holiday Inn Express & Suites Milledgeville.
--------------------------
Retrieving results for Index 64: Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
--------------------------
Retrieving results for Index 75: Atuona.
Closest hotel in Atuona is Hôtel Hiva Oa Hanakee Lodge.
--------------------------
Retrieving results for Index 120: Bogalusa.
Closest hotel in Bogalusa is Magnolia Garden Inn & Suites.
--------------------------
Retrieving results for Index 134: Lumberton.
Closest hotel in Lumberton is Comfort Inn & Suites.
--------------------------
Retrieving results for Index 192: Prome.
Closest hotel in Prome is Mingalar Garden Hotel.
--------------------------
Retrieving results for Index 196: Shangrao.
Closest hotel in Shangrao is Longtan 

In [35]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
31,Cabo San Lucas,MX,22.8909,-109.9124,Bahia Hotel & Beach House
56,Milledgeville,US,33.0801,-83.2321,Holiday Inn Express & Suites Milledgeville
64,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
75,Atuona,PF,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
120,Bogalusa,US,30.7910,-89.8487,Magnolia Garden Inn & Suites
134,Lumberton,US,34.6182,-79.0086,Comfort Inn & Suites
192,Prome,MM,18.8167,95.2167,Mingalar Garden Hotel
196,Shangrao,CN,28.4532,117.9686,Longtan Lake Hotel
258,Statesboro,US,32.4488,-81.7832,SpringHill Suites by Marriott Statesboro Unive...
299,Pozo Colorado,PY,-23.4923,-58.7972,Hotel Pozo Colorado


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the hotel layer to the map
fig.add_layer(markers)

plt.savefig("./Images/Heatmap with Ideal Cities and Hotels.png")

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>